In [183]:
import pandas as pd
import numpy as np
#import geopandas as gpd
import matplotlib.pyplot as plt
import datetime as dt
import json
import ast


Cargamos las bases de datos

In [184]:
dir_data = "../airbnb_final_csv/"
calendar = pd.read_csv(f"{dir_data}calendar_fin.csv")
listings = pd.read_csv(f"{dir_data}listings_fin.csv")
neighbourhoods = pd.read_csv(f"{dir_data}neighbourhoods_fin.csv")
reviews = pd.read_csv(f"{dir_data}reviews_fin.csv")
reviews_gz = pd.read_csv(f"{dir_data}reviews_gz_fin.csv")
listings_gz = pd.read_csv(f"{dir_data}listings_gz_fin.csv")

/tmp/ipykernel_10450/473357094.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  calendar = pd.read_csv(f"{dir_data}calendar_fin.csv")
/tmp/ipykernel_10450/473357094.py:7: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  listings_gz = pd.read_csv(f"{dir_data}listings_gz_fin.csv")


Procesamos los datos para poder cargarlos a la base de datos. Empezamos con dos datos de listings_gz

In [185]:
# Proceso las columnas que deberian que son booleanas y tienen "f" y "t" como valor:

listings_gz = listings_gz.drop_duplicates()

listings_gz["host_is_superhost"] = listings_gz["host_is_superhost"].replace({"f":"false", "t":"true", })
listings_gz["host_has_profile_pic"] = listings_gz["host_has_profile_pic"].replace({"f":"false", "t":"true", })
listings_gz["host_identity_verified"] = listings_gz["host_identity_verified"].replace({"f":"false", "t":"true", })
listings_gz["has_availability"] = listings_gz["has_availability"].replace({"f":"false", "t":"true", })
listings_gz["instant_bookable"] = listings_gz["instant_bookable"].replace({"f":"false", "t":"true", })

# Tratamos los valores nulos:

# Calendar_updated podriamos eliminarlo ya que no tiene ningun valor, todos son valores nulos
listings_gz["calendar_updated"].fillna(0).astype(int)

0        0
1        0
2        0
3        0
4        0
        ..
93596    0
93597    0
93598    0
93599    0
93600    0
Name: calendar_updated, Length: 93601, dtype: int64

Ahora vamos con las variables numericas

In [186]:
# Empezamos quitandole el simbolo del porcentaje y diviendiendo entre 100 a las variables host_response_rate y a host_
# Para ello utilizamos una funcion:

def conversion_porcentajes(name):
    name = str(name).replace("%","")
    name = float(name)/100
    return name

def quitar_dolar(name):
    name = str(name).replace("$", "").replace(",", "")
    name = float(name)
    return name

listings_gz["host_response_rate"] = listings_gz["host_response_rate"].apply(conversion_porcentajes)
listings_gz["host_acceptance_rate"] = listings_gz["host_acceptance_rate"].apply(conversion_porcentajes)
listings_gz["price"] = listings_gz["price"].apply(quitar_dolar)

# Convertimos algunos float en int, antes quitamos valores nulos igualando a 0 (ya que solo hay 6 valores nulos)

listings_gz["host_listings_count"] = listings_gz["host_listings_count"].fillna(0).astype(int)
listings_gz["host_total_listings_count"] = listings_gz["host_total_listings_count"].fillna(0).astype(int)

listings_gz["accommodates"] = listings_gz["accommodates"].fillna(0).astype(int)
listings_gz["bedrooms"] = listings_gz["bedrooms"].fillna(0).astype(int)
listings_gz["beds"] = listings_gz["beds"].fillna(0).astype(int)

listings_gz["minimum_minimum_nights"] = listings_gz["minimum_minimum_nights"].fillna(0).astype(int)
listings_gz["maximum_minimum_nights"] = listings_gz["maximum_minimum_nights"].fillna(0).astype(int)
listings_gz["minimum_maximum_nights"] = listings_gz["minimum_maximum_nights"].fillna(0).astype(int)
listings_gz["maximum_maximum_nights"] = listings_gz["maximum_maximum_nights"].fillna(0).astype(int)


# Nos aseguramos de que host_verifications este en formato json, pero antes manejamos los valores nulos

listings_gz['host_verifications'] = listings_gz['host_verifications'].fillna('[ ]')
listings_gz['host_verifications'] = listings_gz['host_verifications'].apply(lambda x: json.dumps(ast.literal_eval(x)))

In [187]:
# Create dataframes from SQL tables
# Calendar table
proc_calendar = pd.DataFrame(columns=[
    'listing_id', 'date', 'available', 'price', 'adjusted_price', 'minimum_nights', 'maximum_nights'
])

# Property type table
proc_property_type = pd.DataFrame(columns=['type'])

# Listings table 
proc_listings = pd.DataFrame(columns=[
    'id', 'name', 'host_id', 'neighbourhood_id', 'city_id', 'latitude', 'longitude',
    'room_type', 'property_type', 'price', 'minimum_nights', 'maximum_nights', 'license'
])

# Listings description table
proc_listings_description = pd.DataFrame(columns=[
    'listings_id', 'description', 'picture_url', 'accommodates', 'bathrooms', 'bathrooms_text',
    'bedrooms', 'beds', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights',
    'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm'
])

# Listings amenities table
proc_listings_amenities = pd.DataFrame(columns=['listing_id', 'amenity'])

# Listings availability table
proc_listings_availability = pd.DataFrame(columns=[
    'listing_id', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights',
    'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated',
    'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365',
    'instant_bookable'
])

# Listings reviews table
proc_listings_reviews = pd.DataFrame(columns=[
    'listing_id', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
    'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy',
    'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication',
    'review_scores_location', 'review_scores_value', 'reviews_per_month'
])

# Neighborhood groups table
proc_neighborhood_groups = pd.DataFrame(columns=['id', 'name'])

# Neighborhoods table
proc_neighborhoods = pd.DataFrame(columns=[
    'id', 'name', 'neighborhood_group_id', 'city_id'
])

# Countries table
proc_countries = pd.DataFrame(columns=['iso_3166_1', 'name'])

# Cities table
proc_cities = pd.DataFrame(columns=['id', 'name', 'country_code'])

# Hosts table
proc_hosts = pd.DataFrame(columns=[
    'id', 'url', 'name', 'host_since', 'location', 'host_about', 'response_time', 'response_rate',
    'acceptance_rate', 'is_superhost', 'thumbnail_url', 'picture_url', 'neighborhood',
    'listings_count', 'total_listings_count', 'verifications', 'has_profile_pic',
    'identity_verified', 'calculated_host_listings_count',
    'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms',
    'calculated_host_listings_count_shared_rooms'
])


# Users table
proc_users = pd.DataFrame(columns=['id', 'name'])

# Reviews table
proc_reviews = pd.DataFrame(columns=['id', 'listing_id', 'date', 'reviewer_id', 'comments'])

In [188]:
# Create a dictionary to map information from listings to the new dataframes
listings_info = {
    'proc_listings_description': {
        'listings_id': listings_gz['id'],
        'description': listings_gz['description'],
        'picture_url': listings_gz['picture_url'],
        'accommodates': listings_gz['accommodates'],
        'bathrooms': listings_gz['bathrooms'],
        'bathrooms_text': listings_gz['bathrooms_text'],
        'bedrooms': listings_gz['bedrooms'],
        'beds': listings_gz['beds'],
        'minimum_minimum_nights': listings_gz['minimum_minimum_nights'],
        'maximum_minimum_nights': listings_gz['maximum_minimum_nights'],
        'minimum_maximum_nights': listings_gz['minimum_maximum_nights'],
        'maximum_maximum_nights': listings_gz['maximum_maximum_nights'],
        'minimum_nights_avg_ntm': listings_gz['minimum_nights_avg_ntm'],
        'maximum_nights_avg_ntm': listings_gz['maximum_nights_avg_ntm']
    },
    'proc_listings': {
        'id': listings_gz['id'],
        'name': listings_gz['name'],
        'host_id': listings_gz['host_id'],
        'neighbourhood_id': listings_gz['neighbourhood_cleansed'],
        'city_id': listings_gz['city'],
        'latitude': listings_gz['latitude'],
        'longitude': listings_gz['longitude'],
        'room_type': listings_gz['room_type'],
        'property_type': listings_gz['property_type'],
        'price': listings_gz['price'],
        'minimum_nights': listings_gz['minimum_nights'],
        'maximum_nights': listings_gz['maximum_nights'],
        'license': listings_gz['license']
    }, 
    'proc_listings_amenities': {
        'listing_id': listings_gz['id'],
        'amenity': listings_gz['amenities']
    }, 
    'proc_host': {
        'id': listings_gz['host_id'],
        'url': listings_gz['host_url'],
        'name': listings_gz['host_name'],
        'host_since': listings_gz['host_since'],
        'location': listings_gz['host_location'],
        'host_about': listings_gz['host_about'],
        'response_time': listings_gz['host_response_time'],
        'response_rate': listings_gz['host_response_rate'],
        'acceptance_rate': listings_gz['host_acceptance_rate'],
        'is_superhost': listings_gz['host_is_superhost'],
        'thumbnail_url': listings_gz['host_thumbnail_url'],
        'picture_url': listings_gz['host_picture_url'],
        'neighborhood': listings_gz['host_neighbourhood'],
        'listings_count': listings_gz['host_listings_count'],
        'total_listings_count': listings_gz['host_total_listings_count'],
        'verifications': listings_gz['host_verifications'],
        'has_profile_pic': listings_gz['host_has_profile_pic'],
        'identity_verified': listings_gz['host_identity_verified'],
        'calculated_host_listings_count': listings_gz['calculated_host_listings_count'],
        'calculated_host_listings_count_entire_homes': listings_gz['calculated_host_listings_count_entire_homes'],
        'calculated_host_listings_count_private_rooms': listings_gz['calculated_host_listings_count_private_rooms'],
        'calculated_host_listings_count_shared_rooms': listings_gz['calculated_host_listings_count_shared_rooms']
    }, 
    'proc_listings_availability': {
        'listing_id': listings_gz['id'],
        'minimum_minimum_nights': listings_gz['minimum_minimum_nights'],
        'maximum_minimum_nights': listings_gz['maximum_minimum_nights'],
        'minimum_maximum_nights': listings_gz['minimum_maximum_nights'],
        'maximum_maximum_nights': listings_gz['maximum_maximum_nights'],
        'minimum_nights_avg_ntm': listings_gz['minimum_nights_avg_ntm'],
        'maximum_nights_avg_ntm': listings_gz['maximum_nights_avg_ntm'],
        'calendar_updated': listings_gz['calendar_updated'],
        'has_availability': listings_gz['has_availability'],
        'availability_30': listings_gz['availability_30'],
        'availability_60': listings_gz['availability_60'],
        'availability_90': listings_gz['availability_90'],
        'availability_365': listings_gz['availability_365'],
        'instant_bookable': listings_gz['instant_bookable']
    },
    'proc_listings_reviews': {
        'listing_id': listings_gz['id'],
        'number_of_reviews': listings_gz['number_of_reviews'],
        'number_of_reviews_ltm': listings_gz['number_of_reviews_ltm'],
        'number_of_reviews_l30d': listings_gz['number_of_reviews_l30d'],
        'first_review': listings_gz['first_review'],
        'last_review': listings_gz['last_review'],
        'review_scores_rating': listings_gz['review_scores_rating'],
        'review_scores_accuracy': listings_gz['review_scores_accuracy'],
        'review_scores_cleanliness': listings_gz['review_scores_cleanliness'],
        'review_scores_checkin': listings_gz['review_scores_checkin'],
        'review_scores_communication': listings_gz['review_scores_communication'],
        'review_scores_location': listings_gz['review_scores_location'],
        'review_scores_value': listings_gz['review_scores_value'],
        'reviews_per_month': listings_gz['reviews_per_month']
    }, 
    'proc_property_type': {
        'type': listings_gz['property_type']
    }
    
}
# Map the dictionary to the new dataframes
for key, value in listings_info.items():
    for col, series in value.items():
        if key == 'proc_listings_description':
            proc_listings_description[col] = series
        elif key == 'proc_listings':
            proc_listings[col] = series
        elif key == 'proc_listings_amenities':
            proc_listings_amenities[col] = series
        elif key == 'proc_host':
            proc_hosts[col] = series
        elif key == 'proc_listings_availability':
            proc_listings_availability[col] = series
        elif key == 'proc_listings_reviews':
            proc_listings_reviews[col] = series
        elif key == 'proc_property_type':
            proc_property_type[col] = series
            

# Create a dictionary to map information from reviews to the new dataframes
reviews_info = {
    'proc_reviews': {
        'id': reviews_gz['id'],
        'listing_id': reviews_gz['listing_id'],
        'date': reviews_gz['date'],
        'reviewer_id': reviews_gz['reviewer_id'],
        'comments': reviews_gz['comments']
    }, 
    'proc_users': {
        'id': reviews_gz['reviewer_id'],
        'name': reviews_gz['reviewer_name']
    }
}

# Map the dictionary to the new dataframes
for key, value in reviews_info.items():
    for col, series in value.items():
        if key == 'proc_reviews':
            proc_reviews[col] = series
        elif key == 'proc_users':
            proc_users[col] = series

# Create a dictionary to map information from calendar to the new dataframes
calendar_info = {
    'proc_calendar': {
        'listing_id': calendar['listing_id'],
        'date': calendar['date'],
        'available': calendar['available'],
        'price': calendar['price'],
        'adjusted_price': calendar['adjusted_price'],
        'minimum_nights': calendar['minimum_nights'],
        'maximum_nights': calendar['maximum_nights']
    }
}
# Map the dictionary to the new dataframes
for key, value in calendar_info.items():
    for col, series in value.items():
        if key == 'proc_calendar':
            proc_calendar[col] = series

neighbourhoods_info = {
    'proc_neighborhoods': {
        'name': neighbourhoods['neighbourhood'].unique(),
        'neighborhood_group_id': neighbourhoods['neighbourhood_group'],
        'city_id': neighbourhoods['city']
    }
}

# Map the dictionary to the new dataframes
for key, value in neighbourhoods_info.items():
    for col, series in value.items():
        if key == 'proc_neighborhoods':
            proc_neighborhoods[col] = series
    

In [189]:
#Con esto renombro las columnas
proc_hosts = proc_hosts.drop_duplicates(subset="id")


# Esto es una prueba para ver si sale bien
proc_hosts.to_csv('proc_airbnb/proc_host.csv', index=False)

In [199]:
proc_listings_availability = proc_listings_availability.drop_duplicates(subset="listing_id")
proc_listings_description = proc_listings_description.drop_duplicates(subset="listings_id")

proc_listings_amenities.to_csv('proc_airbnb/proc_listings_amenities.csv', index=False)
proc_listings_description.to_csv('proc_airbnb/proc_listings_description.csv', index=False)
proc_listings_availability.to_csv('proc_airbnb/proc_listings_availability.csv', index=False)
proc_listings_reviews.to_csv('proc_airbnb/proc_listings_reviews.csv', index=False)

In [191]:
proc_property_type = proc_property_type.drop_duplicates(subset="type")
proc_property_type.to_csv('proc_airbnb/proc_property_type.csv', index=False)

In [192]:
cities = ['barcelona', 'madrid', 'malaga', 'mallorca', 'menorca', 'sevilla', 'valencia']
proc_cities['id'] = range(1, len(cities) + 1)
proc_cities['name'] = cities
proc_cities['country_code'] = 'ES'

proc_cities.to_csv('proc_airbnb/proc_cities.csv', index=False)

In [193]:
proc_countries['iso_3166_1'] = np.array(['ES'])
proc_countries['name'] = np.array(['Spain'])

proc_countries.to_csv('proc_airbnb/proc_countries.csv', index=False)

In [194]:
proc_neighborhood_groups['name'] = neighbourhoods['neighbourhood_group'].unique()
null = proc_neighborhood_groups[proc_neighborhood_groups['name'].isnull()].index
proc_neighborhood_groups.drop(null, inplace=True)
proc_neighborhood_groups['id'] = range(1, len(proc_neighborhood_groups) + 1)

proc_neighborhood_groups.to_csv('proc_airbnb/proc_neighborhood_groups.csv', index=False)

In [195]:
proc_listings['neighbourhood_id'] = proc_listings['neighbourhood_id'].map(dict(proc_neighborhoods[['name', 'id']].values))

proc_listings = proc_listings.drop_duplicates(subset="id")
proc_listings.to_csv('proc_airbnb/proc_listings.csv', index=False)

In [201]:
proc_reviews = proc_reviews.drop_duplicates(subset="id")
proc_reviews.to_csv('proc_airbnb/proc_reviews.csv', index=False)

In [203]:
proc_users = proc_users.drop_duplicates(subset="id")
proc_users.to_csv('proc_airbnb/proc_users.csv', index=False)

In [196]:
proc_calendar['date'] = pd.to_datetime(proc_calendar['date'], format='%Y-%m-%d')

boolean_map = {'t': True, 'f': False}
proc_calendar['available'] = proc_calendar['available'].map(boolean_map)

proc_calendar['price'] = proc_calendar['price'].str.replace('$', '').str.replace(',', '').astype(float)
proc_calendar['adjusted_price'] = proc_calendar['adjusted_price'].str.replace('$', '').str.replace(',', '').astype(float)

proc_calendar['maximum_nights'] = proc_calendar['maximum_nights'].fillna(0).astype(int)
proc_calendar['minimum_nights'] = proc_calendar['minimum_nights'].fillna(0).astype(int)

In [197]:
# proc_calendar.to_csv('proc_airbnb/proc_calendar.csv', index=False)